In [1]:
import pandas as pd
import torch
import torch.nn as nn
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from utils import sample_sequence
from torch.autograd import Variable
from DataLoader import *
device = torch.device("mps")

/Users/tschip/workspace/baa/.venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/tschip/workspace/baa/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
pandas.read_csv('data/gpt2_input.csv', sep=',')

,players_name,team_id,league_id,season,games_played,games_lineups,games_minutes,games_position,captain,substitutes_in,substitutes_out,substitutes_bench,goals_total,cards_yellow,cards_yellowred,cards_red
0,M. Domizzi,494,135,2011,31,30,2598,Defender,False,1,5,2,1,7,0,0
1,Damiano Ferronetti,494,135,2011,15,11,1080,Defender,False,4,1,8,1,2,1,0
2,Michele Pazienza,494,135,2011,15,12,1187,Midfielder,False,3,3,6,1,3,0,0
3,A. Sissoko,494,135,2011,0,0,0,Midfielder,False,0,0,5,0,0,0,0
4,Neuton Sérgio Piccoli,494,135,2011,4,1,120,Defender,False,3,0,17,0,0,0,0
5,Jean-Alain Fanchone,494,135,2011,0,0,0,Defender,False,0,0,1,0,0,0,0
6,K. Asamoah,494,135,2011,31,31,2728,Defender,False,0,3,0,3,3,0,0
7,M. Benatia,494,135,2011,27,27,2381,Defender,False,0,1,0,1,9,0,0
8,Emmanuel Agyemang-Badu,494,135,2011,10,6,607,Midfielder,False,4,0,9,0,2,0,0
9,T. Doubaï,494,135,2011,1,0,1,Midfielder,False,1,0,11,0,0,0,0


In [14]:
print(torch.backends.mps.is_available()) #the MacOS is higher than 12.3+
print(torch.backends.mps.is_built())

True
True


In [15]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')
model = nn.DataParallel(model)

In [16]:
model.to(device)

DataParallel(
  (module): GPT2LMHeadModel(
    (transformer): GPT2Model(
      (wte): Embedding(50257, 768)
      (wpe): Embedding(1024, 768)
      (drop): Dropout(p=0.1, inplace=False)
      (h): ModuleList(
        (0-11): 12 x GPT2Block(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): GPT2Attention(
            (c_attn): Conv1D()
            (c_proj): Conv1D()
            (attn_dropout): Dropout(p=0.1, inplace=False)
            (resid_dropout): Dropout(p=0.1, inplace=False)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): GPT2MLP(
            (c_fc): Conv1D()
            (c_proj): Conv1D()
            (act): NewGELUActivation()
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
      )
      (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    )
    (lm_head): Linear(in_features=768, out_features=50257, bias=False)
  )
)

In [17]:
dataset = GPTTableDatabase(None, None, None, tokenizer, 5, 800)
model.load_state_dict(torch.load('models/GPT_ep19.pt', map_location=device))
model.eval()

DataParallel(
  (module): GPT2LMHeadModel(
    (transformer): GPT2Model(
      (wte): Embedding(50257, 768)
      (wpe): Embedding(1024, 768)
      (drop): Dropout(p=0.1, inplace=False)
      (h): ModuleList(
        (0-11): 12 x GPT2Block(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): GPT2Attention(
            (c_attn): Conv1D()
            (c_proj): Conv1D()
            (attn_dropout): Dropout(p=0.1, inplace=False)
            (resid_dropout): Dropout(p=0.1, inplace=False)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): GPT2MLP(
            (c_fc): Conv1D()
            (c_proj): Conv1D()
            (act): NewGELUActivation()
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
      )
      (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    )
    (lm_head): Linear(in_features=768, out_features=50257, bias=False)
  )
)

In [20]:
list_of_hypothesis = []
results = {}
with torch.no_grad():
    batch = dataset.get_data(1, 'test')
    input_string = batch[-1]
    batch = batch[:-1]
    print(f'input_string: {input_string}')
    print(f'batch: {batch}')
    #idx = dataset.get_idx('test')
    #references = dataset.get_reference(idx, 'test')
    #table_id = dataset.get_table_id(idx, 'test')
    results['players stats'] = []

    batch = tuple(Variable(t).to(device) for t in batch)
    trg_inp, trg_out, mask, caption = batch

    fake_inputs = caption

    samples = sample_sequence(model, 100, fake_inputs, [], top_k=3)

    samples = samples[:, caption.shape[1]:]
    samples = samples.cpu().data.numpy()
    print(f'samples len: {len(samples)}')
    for s in samples:
        text = tokenizer.decode(s, clean_up_tokenization_spaces=True)
        text = text[: text.find(tokenizer.eos_token)]
        results['players stats'].append(text)

        hypothesis = text.lower().split(' ')
        
        list_of_hypothesis.append(hypothesis)

['Given', 'Ġthe', 'Ġtable', 'Ġtitle', 'Ġ"', 'players', ':', 'Ġgames', 'Ġplayed', 'Ġand', 'Ġtotal', 'Ġgoals', 'Ġscored', 'Ġin', 'Ġthe', 'Ġseason', '"', 'Ġ.', 'Ġ', 'In', 'Ġrow', 'Ġ1', 'Ġ,', 'Ġthe', 'Ġplayers', '_', 'name', 'Ġis', 'ĠM', '.', 'ĠDom', 'izz', 'i', 'Ġ,', 'Ġthe', 'Ġseason', 'Ġis', 'Ġ2011', 'Ġ,', 'Ġthe', 'Ġgames', '_', 'played', 'Ġis', 'Ġ31', 'Ġ,', 'Ġthe', 'Ġgoals', '_', 'total', 'Ġis', 'Ġ1', 'Ġ.', 'ĠIn', 'Ġrow', 'Ġ2', 'Ġ,', 'Ġthe', 'Ġplayers', '_', 'name', 'Ġis', 'ĠDam', 'iano', 'ĠFer', 'ron', 'etti', 'Ġ,', 'Ġthe', 'Ġseason', 'Ġis', 'Ġ2011', 'Ġ,', 'Ġthe', 'Ġgames', '_', 'played', 'Ġis', 'Ġ15', 'Ġ,', 'Ġthe', 'Ġgoals', '_', 'total', 'Ġis', 'Ġ1', 'Ġ.', 'ĠIn', 'Ġrow', 'Ġ3', 'Ġ,', 'Ġthe', 'Ġplayers', '_', 'name', 'Ġis', 'ĠMichele', 'ĠP', 'az', 'ien', 'za', 'Ġ,', 'Ġthe', 'Ġseason', 'Ġis', 'Ġ2011', 'Ġ,', 'Ġthe', 'Ġgames', '_', 'played', 'Ġis', 'Ġ15', 'Ġ,', 'Ġthe', 'Ġgoals', '_', 'total', 'Ġis', 'Ġ1', 'Ġ.', 'ĠIn', 'Ġrow', 'Ġ4', 'Ġ,', 'Ġthe', 'Ġplayers', '_', 'name', 'Ġis', 'ĠA', '.', 

In [23]:
results['players stats'][0]

'in the 2011 season of games played, the only time m. domizzi scored more than 1 goals was in. 2011.'

Loss Implementation

In [4]:
import numpy as np
import nltk
from nltk import word_tokenize
import ot

nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/tschip/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [24]:
# Example input table and decoded target sequence
table = input_string#"Seattle, Washington | Skokie, Illinois"
generated_text = results['players stats'][0]#"Seattle, Washington; Skokie, Illinois"

# Tokenize and extract nouns using NLTK
def extract_nouns(text):
    tokens = word_tokenize(text)
    pos_tags = nltk.pos_tag(tokens)
    nouns = [word for word, pos in pos_tags if pos.startswith('N')]
    return nouns

table_nouns = extract_nouns(table)
generated_text_nouns = extract_nouns(generated_text)

# Define cost matrix based on Jaccard similarity (you can use other similarity measures)
cost_matrix = np.zeros((len(table_nouns), len(generated_text_nouns)))

for i, noun_table in enumerate(table_nouns):
    for j, noun_generated in enumerate(generated_text_nouns):
        # Use Jaccard similarity as the cost (you can use other similarity measures)
        cost_matrix[i, j] = len(set(noun_table).intersection(noun_generated)) / len(set(noun_table).union(noun_generated))

# Normalize the cost matrix to make it a true cost matrix
cost_matrix /= cost_matrix.sum()

# Solve the Optimal Transport problem
optimal_transport_plan = ot.emd(np.ones(len(table_nouns))/len(table_nouns),
                                np.ones(len(generated_text_nouns))/len(generated_text_nouns),
                                cost_matrix)

# Compute the OT loss
ot_loss = np.sum(optimal_transport_plan * cost_matrix)

print("Optimal Transport Loss:", ot_loss)


Optimal Transport Loss: 0.0004165982716259307
